# 머신러닝 with Python 
## 09 앙상블 학습 보팅 실습

##### 보팅 알고리즘을 활용해 꽃 데이터를 분류하는 모형을 생성해본다.

책 p.259~

---

하단부에서는 모델별 voting weight 값을 변경해보면서 실험해 보았다.

> 배운점1: voting 방식을 soft로 하려면 결과가 예측값이 아니라 예측 확률이 나와야 한다. (분류기에서 안됨)

>  배운점2: 같은 분류기들로 이루어진 앙상블이라도 weight 값을 변경하면 정확도에 영향을 미칠 수 있다. 


### 데이터 불러오기

In [1]:
from sklearn import datasets 

In [2]:
raw_iris = datasets.load_iris() # 꽃 데이터 가져오기

In [3]:
# 데이터 살펴보기

raw_iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

### 피처, 타깃 데이터 지정

보통 train data는 행렬이므로 대문자 X, 예측되어야 할 값은 스칼라로 소문자 y가 된다는 것을 들은게 생각이 나부렀다.}

In [4]:
X = raw_iris.data
y = raw_iris.target

### 트레이닝/테스트 데이터 분할

In [6]:
from sklearn.model_selection import train_test_split              # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 분리. randomstate는 고정

### 데이터 표준화

In [7]:
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                     # 표준화 스케일러 지정
std_scale.fit(X_tn)                              # 트레이닝 피처를 기준으로 표준화를 적합시키기


X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)             # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습 - 보팅 학습

이번 실습에서는 로지스틱 회귀분석, 가우시안 나이브 베이즈, 서포트 벡터 머신을 사용한다.

In [9]:
from sklearn.linear_model import LogisticRegression # 로지트틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])

### 데이터 예측

In [10]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


### 정확도 평가

In [11]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

0.9736842105263158


### confusion matrix 확인

각 클래스를 각 클래스로 예측한 개수. 대부분 데이터가 잘 분리된 것을 볼 수 있다.

In [12]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


### 분류 리포트 확인

In [14]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



## 실험1- voting 방식 변경해보기 (soft로) -실패

### 데이터 학습 - 보팅 학습

로지스틱 회귀분석, 가우시안 나이브 베이즈, 서포트 벡터 머신을 그대로 사용한다.

In [15]:
from sklearn.linear_model import LogisticRegression # 로지트틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'soft',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 voting='soft', weights=[1, 1, 1])

### 데이터 예측

In [16]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

AttributeError: predict_proba is not available when  probability=False

아예 돌아가지 않는다.

$ AttributeError: predict_proba is not available when  probability=False $

아마도 분류가 된 모형에서 확률을 찾으려니까 안되는 것인가? 추측해본다.

왜그럴까?

---

> *그런데 제가 제대로 알고 있는 유일한 모델인 Decision Tree는 답을 확률로 알려주는 것이 아니니 Soft Voting이 불가능할까요? 놀랍게도 scikit learn에서 제공하는 DecisionTreeClassifier에는 각 class의 확률을 계산해주는  predict_prob 함수가 있습니다. 확률을 계산하는 방법은 단순하게 train dataset(의 일부) 를 Decision Tree에 넣어보고 각 leaf로 나오는 값들 중 정확한 값인 비율을 계산합니다* *참조: https://devkor.tistory.com/entry/Soft-Voting-%EA%B3%BC-Hard-Voting*

라는 말이 있다. 이런 부분을 미루어 보았을 떄 위에서 사용한 모델들에서의 결과는 

```
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)
```
라고 했을 때의 결과처럼 아예 어떤 클래스에 속하는지 라벨을 뱉는다. 아마 그 부분의 문제인 것 같다.

실제로 위 글에서도 'Decision Tree Classifier의 Probability 계산'이라는 제목의 문단에 코드로 이렇게 적혀있다

```
from sklearn.tree import DecisionTreeClassifier
iowa_model2 = DecisionTreeClassifier()
iowa_model2.fit(X, y)
predictions = iowa_model2.predict_proba(X)
print(predictions.tolist())
```

predict_prob 함수를 사용해서 확률을 계산한다고 했는데... 생각한 이유가 얼추 맞는 것 같다는 느낌이다.

## 실험2- weight 값 변경하기
(lr = 2, svm = 1, gnb = 1)

### 데이터 학습

In [17]:
from sklearn.linear_model import LogisticRegression # 로지트틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [2, 1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[2, 1, 1])

### 데이터 예측

In [18]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


### 정확도 평가

In [19]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

0.9736842105263158


### confusion matrix 확인

각 클래스를 각 클래스로 예측한 개수. 대부분 데이터가 잘 분리된 것을 볼 수 있다.

In [20]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


### 분류 리포트 확인

In [21]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



## 실험3- weight 값 변경하기
(lr = 1, svm = 3, gnb = 1)

### 데이터 학습

In [22]:
from sklearn.linear_model import LogisticRegression # 로지트틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 3, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 3, 1])

### 데이터 예측

In [23]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


### 정확도 평가

In [24]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

0.9736842105263158


### confusion matrix 확인

각 클래스를 각 클래스로 예측한 개수. 대부분 데이터가 잘 분리된 것을 볼 수 있다.

In [25]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


### 분류 리포트 확인

In [26]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



너무 단순한 데이터로 한게 문제인 걸까... 다른게 없네.

## 실험4- weight 값 변경하기(정확도 100 나온 사례)
(lr = 1, svm = 1, gnb = 8)

### 데이터 학습

In [27]:
from sklearn.linear_model import LogisticRegression # 로지트틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1, 8])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 8])

### 데이터 예측

In [28]:
pred_voting = clf_voting.predict(X_te_std)          # std된 테스트 피처 데이터를 넣고 실행
print(pred_voting)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1]


### 정확도 평가

In [29]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_voting)        # 실제값과 예측값을 넣음
print(accuracy)

1.0


### confusion matrix 확인

각 클래스를 각 클래스로 예측한 개수. 대부분 데이터가 잘 분리된 것을 볼 수 있다.

In [30]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_voting)
print(conf_matrix)

[[13  0  0]
 [ 0 16  0]
 [ 0  0  9]]


### 분류 리포트 확인

In [31]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_voting)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00         9

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



정확도가 100이 나오셨어요....?.....??????????

직감적으로 100%라는 것은 굉장히 안좋은 숫자라고 느껴진다. 다른 데이터가 들어오면 정확도가 순식간에 떨어질 것이 불보듯 뻔하다고 해야하나. 재미있는 실험이었다.